In [20]:
# I am making it a new tradition for me to check sys.executable to ensure that I am working in the correct environment.
# For the love of god, I can feel more of my bare scalp every week.
import sys
sys.executable

'/home/dave/anaconda3/bin/python'

In [32]:
import json
import pandas as pd
import tqdm
from tqdm import tqdm

In [6]:
data_dir = '/media/dave/Prepper/ol_dump_2025-12-31.txt'

In [3]:
rows = []

In [24]:
total_lines = sum(1 for _ in open(data_dir, "r", encoding="utf-8"))
print(f"Total Lines: {total_lines:,}")

Total Lines: 116,417,996


In [25]:
total_lines = 116417996

In [ ]:
with open(data_dir, "r", encoding='utf-8') as f:
    for line in tqdm(f, total=total_lines, desc="Scanning dump"):
        if not line.startswith("/type/work"):
            continue

        parts = line.rstrip("\n").split("\t")
        data = json.loads(parts[4])

        if "description" in data: 
            new_row = (data["title"], data["description"])
            rows.append(new_row)
            
    #    extracted_row = 
    #    rows.append(extracted_row)

Scanning dump:   4%|▍         | 4908774/116417996 [00:36<18:05, 102712.20it/s]